In [1]:
from IPython.display import display
from ipywidgets import interact
import numpy as np
import pandas as pd
from simple.backtest import npBacktestMarket, getProfit
from simple.funcs import tickSpeed
from simple.geneopt import GridOpt
from simple.pretty import pp

In [2]:
X = np.load('/tmp/BTCUSDT.npz')
X

In [3]:
A = X['Ask'].copy()
B = X['Bid'].copy()
C = X['C'].copy().view(np.recarray)
del X
A, B, C

(array([43759.46875  , 43759.4609375, 43758.8515625, ...,     0.       ,
            0.       ,     0.       ]),
 array([43759.4609375, 43756.328125 , 43758.828125 , ...,     0.       ,
            0.       ,     0.       ]),
 rec.array([('2022-01-13T12:34:25.790999',        0, 43760.825, 4633001,  -8., 174., -182., 29, 13, 16),
            ('2022-01-13T12:34:29.274000',        1, 43759.465, 3483001, 320., 371.,  -51., 13,  7,  6),
            ('2022-01-13T12:34:31.683000',        2, 43756.335, 2396000,  89., 152.,  -63., 14,  7,  7),
            ...,
            ('2022-07-29T23:59:59.996999', 72059785, 23773.745,    1999,  -5.,   0.,   -5.,  4,  0,  4),
            ('2022-07-29T23:59:59.997999', 72059786, 23777.275,       0,   5.,   5.,    0.,  2,  2,  0),
            ('2022-07-29T23:59:59.997999', 72059787, 23775.705,       0,  -1.,   0.,   -1.,  1,  0,  1)],
           dtype=[('DateTime', '<M8[us]'), ('Index', '<i8'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('Buy

In [4]:
np.power(2, 25)

33554432

In [5]:
def model(Threshold: int=(2, 5, 0.125), Period: int=(500, 15000, 500), Hold: int=(1, 25, 1)):
    Speed = tickSpeed(C, Period, log=True)
    D = npBacktestMarket(C.DateTime, A, B, Speed, Threshold, hold=np.power(2, Hold)*1_000_000)
    P = getProfit(D)
    return P.MidPnL.sum(), {
        'Count': len(P),
        'AvgMid': P.MidPnL.mean() if len(P) > 0 else 0,
        'RawPnL': P.RawPnL.sum() if len(P) > 0 else 0,
        'Fee': P.Fee.sum() if len(P) > 0 else 0,
        'MidPnL': P.MidPnL.sum() if len(P) > 0 else 0,
        'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
     }

In [6]:
G = GridOpt(model)
G.fullSearch()
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Fitness')
X.Threshold = X.Threshold.apply(lambda x: f'{x:1.3f}')  # explicit float format to workaround formatting bug
X

  0%|          | 0/18750 [00:00<?, ?it/s]

,Threshold,Period,Hold,Fitness,Count,AvgMid,RawPnL,Fee,MidPnL,Sharpe
2996,2.375,15000,22,-56975.532227,23,-2477.197053,-57262.476562,7.070759e+02,-56975.532227,-21.519115
2997,2.375,15000,23,-56975.532227,23,-2477.197053,-57262.476562,7.070759e+02,-56975.532227,-21.519115
2998,2.375,15000,24,-56975.532227,23,-2477.197053,-57262.476562,7.070759e+02,-56975.532227,-21.519115
2999,2.375,15000,25,-56975.532227,23,-2477.197053,-57262.476562,7.070759e+02,-56975.532227,-21.519115
2973,2.375,14500,24,-54912.627930,21,-2614.887044,-55353.427734,6.600849e+02,-54912.627930,-20.303025
...,...,...,...,...,...,...,...,...,...,...
759,2.125,500,10,39600.779297,34847,1.136419,-329193.138672,1.038385e+06,39600.779297,-19499.114143
757,2.125,500,8,39767.843750,35828,1.109965,-330560.994141,1.066765e+06,39767.843750,-23478.727289
760,2.125,500,11,39772.429688,34516,1.152290,-328703.351562,1.028783e+06,39772.429688,-17994.901185
11,2.000,500,12,41656.293945,38915,1.070443,-356320.853516,1.151426e+06,41656.293945,-19430.129106


In [7]:
X.to_hdf('grid.hdf', key='model21')

In [8]:
@interact(hold=(1,25), column=X.columns[4:])
def Plot(hold, column):
    display(pp(X[X.Hold == hold].pivot(index='Period', columns='Threshold', values=column).astype(int)))

interactive(children=(IntSlider(value=13, description='hold', max=25, min=1), Dropdown(description='column', o…